# Project description

Sweet Lift Taxi company has collected historical data on taxi orders at airports. To attract more drivers during peak hours, we need to predict the amount of taxi orders for the next hour.

The goal RMSE metric on the test set sis less than 48.

## Preparation

We will import our necessary libraries and download our data.

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.stattools import arma_order_select_ic
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

ModuleNotFoundError: No module named 'pmdarima'

In [ ]:
data = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/taxi.csv', parse_dates=['datetime'], index_col='datetime')

We will resample our datetime column to 1 hour segments.

In [ ]:
data = data.resample('1H').sum()
data.head()

## Analysis

We will now analyze the data, looking at trends and seasonanality.

In [ ]:
decomposed = seasonal_decompose(data)
plt.figure(figsize=(8,8))
plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')
plt.tight_layout()
plt.show()

print(decomposed.seasonal.describe())

We can see that the data is trending up towards the later months. It is hard to tell if we have seasonality from this information though. We don't have data for more than a year, so we can't tell if there is monthly or yearly seasonality. There may be daily seasonality, so we will examine that.

In [ ]:
plot_acf(data.dropna(), lags=96)
plt.show()

We can see from this data that we certainly have strong seasonality every 24 hours. This implies that taxi orders increase and spike around the same time each day, so we will make sure to use that seasonality to train our model.

## Training

We will now make our features and then split our data into testing and training sets.

In [ ]:
def create_lag_features(df, lags=[1,2,3]):
    df = df.copy()
    for lag in lags:
        df[f'lag_{lag}'] = df['num_orders'].shift(lag)
    return df

def add_time_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    return df

In [ ]:
data_features = create_lag_features(data)
data_features = add_time_features(data_features)
data_features = data_features.dropna()

In [ ]:
train, test = train_test_split(data_features, shuffle=False, test_size=0.1)

Our data has been prepared, so we will now test various models and parameters to see which is best.

We will start by training an autoregressive model. We will use 24 lags because we found earlier that our seasonality is daily, every 24 hours.

In [ ]:
ar_model = AutoReg(train['num_orders'], lags=24, seasonal=True)
ar_model = ar_model.fit()
start_value = len(train)
end_value = len(train) + len(test) - 1
ar_pred = ar_model.predict(start=start_value, end=end_value, dynamic=False)
plt.plot(ar_pred, color='blue', label='AR Predictions')
plt.plot(test, color='red', label='AR Test')
plt.legend(loc='upper left')
plt.xticks(rotation=90)
plt.show()

In [ ]:
ar_rmse = np.sqrt(mean_squared_error(test['num_orders'], ar_pred))
print(f'RMSE: {ar_rmse:.2f}')

Our RMSE for this model is 68.93. We will test an MA model as well and see which is better.

In [ ]:
res = arma_order_select_ic(y=train['num_orders'], max_ar=3, max_ma=3)
ma_order = res.bic_min_order[1]
ar_order = res.bic_min_order[0]
ma_model = ARIMA(train['num_orders'], order=(0, 0, ma_order))
ma_model = ma_model.fit()
ma_pred = ma_model.predict(start=start_value, end=end_value, dynamic=False)
plt.plot(ma_pred, color='blue', label='MA Predictions')
plt.plot(test, color='red', label='MA Test')
plt.legend(loc='upper left')
plt.xticks(rotation=90)
plt.show()

In [ ]:
ma_rmse = np.sqrt(mean_squared_error(test['num_orders'], ma_pred))
print(f'RMSE: {ma_rmse:.2f}')

The MA model gave us an RMSE of 84.71, not quite as good as the AR model. Finally, we will try an SARIMA model because our data is non-stationary and seasonal.

In [ ]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 24)
sarima_model = SARIMAX(train['num_orders'],
                       order=order,
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False)
sarima_result = sarima_model.fit(disp=False)
n_periods = len(test)
sarima_pred = sarima_result.forecast(steps=n_periods)
sarima_rmse = np.sqrt(mean_squared_error(test['num_orders'], sarima_pred))
print(f'SARIMA RMSE: {sarima_rmse:.2f}')

Our SARIMA Model is our best model with an RMSE of 44.50, which is under our goal of 48, so we will show our final test with that model.

## Testing

In [ ]:
plt.plot(test.index, test['num_orders'], label='Actual', color='red')
plt.plot(test.index, sarima_pred, label='SARIMA Forecast', color='blue')
plt.legend()
plt.title('SARIMA Forecast vs Actual')
plt.xticks(rotation=90)
plt.show()

## Conclusion

Our SARIMA model was our best model, with an RMSE of 44.50. We can see from the plot above that our patterns follow similarly along the trend and seasonality. 